In [ ]:
import os

from serpapi import GoogleSearch
import json
import pandas as pd

def create_search_params(search_key: str,
                         engine: str = "apple_app_store",
                         device: str = "mobile",
                         country: str = "us",
                         lang: str = "en-us",
                         ):
    return {
    'api_key': 'a3a06a53ed09fd1543aca690eca6549abd5454255f438d1763266db50372d743',    # SerpApi search engine
    'engine': engine,
    'term': search_key,         # search query
    'device': device,            # device to get the results  desktop /
    'country': country,                # country for the search
    'lang': lang,                # language for the search
    'disallow_explicit': False,     # disallowing explicit apps
    'num': 200,                      # number of items per page
    'page': 0,                      # pagination
}

# # app_store_results = []
all_results = []
all_terms = []
for i in range(1, 5):
    df = pd.read_parquet(f"../data/serpapi_{i}.parquet")
    all_terms.extend(df['search_key'].unique().tolist())
for search_key in list(set(all_terms)):
    params = create_search_params(search_key=search_key)
    search = GoogleSearch(params)
    page_results = search.get_dict()
    results = (
        pd.json_normalize(page_results['organic_results'])
        .assign(parsed_at=page_results['search_metadata']['created_at'],
                search_key=search_key,
                source="parsed_serpapi"
                )
    )
    all_results.append(results)
all_results = pd.concat(all_results)
all_results.to_parquet("../data/serpapi_7.parquet")

In [ ]:
# Load into bucket

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
os.getenv("S3_ACCESS_SECRET")

'1JAHo4Wd4q2a8JxtVrk7kNJYfLqdNbf86XSC2Ttk'

In [9]:
os.getenv("S3_ACCESS_SECRET")

'1JAHo4Wd4q2a8JxtVrk7kNJYfLqdNbf86XSC2Ttk'

In [19]:
import glob
glob.glob("../data/*")

['../data/serpapi_3.parquet',
 '../data/serpapi_2.parquet',
 '../data/serpapi_1.parquet',
 '../data/serpapi_4.parquet',
 '../data/serpapi_5.parquet',
 '../data/serpapi_7.parquet',
 '../data/serpapi_6.parquet']

In [18]:
os.listdir(f"../data/")

['serpapi_3.parquet',
 'serpapi_2.parquet',
 'serpapi_1.parquet',
 'serpapi_4.parquet',
 'serpapi_5.parquet',
 'serpapi_7.parquet',
 'serpapi_6.parquet']

In [16]:
import boto3
import os
from botocore.exceptions import NoCredentialsError

# Replace with your credentials and values
aws_access_key_id = os.getenv('S3_ACCESS_KEY')
aws_secret_access_key = os.getenv("S3_ACCESS_SECRET")
bucket_name = 'blockchain-defi'
file_index = 7
s3_key = f'generate_data/serpapi_{file_index}.parquet'     # Destination path in S3
local_file_path = f"../data/serpapi_{file_index}.parquet"          # Local file to upload

# Create S3 client
s3 = boto3.client(
    's3',
    region_name='eu-west-2',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

try:
    s3.upload_file(local_file_path, bucket_name, s3_key)
    print(f"✅ Upload successful: {local_file_path} → s3://{bucket_name}/{s3_key}")
except FileNotFoundError:
    print(f"❌ File not found: {local_file_path}")
except NoCredentialsError:
    print("❌ AWS credentials not found or invalid.")
except Exception as e:
    print(f"❌ Upload failed: {e}")

✅ Upload successful: ../data/serpapi_7.parquet → s3://blockchain-defi/generate_data/serpapi_7.parquet


|# Generate data

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random


fake = Faker()

# Function to simulate a row based on column type
def generate_synthetic_row():
    return {
        'app_name': fake.company(),
        'category': random.choice(['Games', 'Productivity', 'Education', 'Health & Fitness', 'Finance']),
        'rating': round(random.uniform(1.0, 5.0), 2),
        'reviews': random.randint(0, 100000),
        'size': f"{random.randint(20, 500)}MB",
        'installs': f"{random.randint(1, 100)}M+",
        'type': random.choice(['Free', 'Paid']),
        'price': round(random.uniform(0.0, 99.99), 2) if random.random() < 0.2 else 0.0,
        'content_rating': random.choice(['Everyone', 'Teen', 'Mature 17+', 'Everyone 10+']),
        'genres': random.choice(['Action', 'Puzzle', 'Tools', 'Education', 'Lifestyle']),
        'last_updated': fake.date(),
        'current_version': f"{random.randint(1,10)}.{random.randint(0,9)}.{random.randint(0,9)}",
        'android_version': random.choice(['4.0 and up', '5.0 and up', '6.0 and up', 'Varies with device'])
    }

# Generate 1000 synthetic rows
synthetic_data = [generate_synthetic_row() for _ in range(1000)]
synthetic_df = pd.DataFrame(synthetic_data)